# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [5]:
# 1. Registra gli ordini in una lista di classi Order
import math
GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = int(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[5]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1][0:-2])
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        if ask_price>self.bid_price:
            return 0
        else:
            return ask_price*self.quantity 
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        if self.Payment(ask_price)==0:
            return 0
        else:
            return self.Payment(ask_price) - production_cost*self.quantity - self.Distance2()*cost_per_km
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Calcola la distanza in km considerando la curvatura terrestre (cfr. https://en.wikipedia.org/wiki/Haversine_formula)
    def Distance2(self, starting_GCS = GCS_Piazza_Leo):
        R=6371
        lat1=starting_GCS[0]
        lon1=starting_GCS[1]
        lat2=self.GCS[0]
        lon2=self.GCS[1]
        phi1=lat1*math.pi/180
        phi2=lat2*math.pi/180
        dphi=(lat2-lat1)*math.pi/180
        dlambda=(lon2-lon1)*math.pi/180
        a=math.sin(dphi/2)**2+math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        return R*2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    file=open(f"{i+1}.txt",'r')
    ord = Order(file.read())
    orders.append(ord)
    file.close()

In [6]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
bid_prices=[]

for i in range(num_orders):
    new=1
    for j in range(len(bid_prices)):
        if orders[i].bid_price==bid_prices[j]:
            new=0
            break
    if new==1:
        bid_prices.append(orders[i].bid_price)
bid_prices.sort()

for j in range(len(bid_prices)):
    profit=0
    for i in range(num_orders):
        profit += orders[i].Profit(bid_prices[j])
    possible_profits.append(profit)
print(possible_profits)

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
for j in range(len(possible_profits)):
    if possible_profits[j]==max_profit:
        best_price=bid_prices[j]
        break
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

[-107670.0, -95676.3000000001, -84563.20000000007, -73944.49999999999, -52359.0, -31267.199999999968, -10373.59999999997, 0.0, 10037.999999999969, 19894.400000000027, 29791.799999999963, 39042.000000000044, 48541.5, 57998.39999999994, 67370.80000000002, 74976.0, 83676.6000000001, 92096.0, 99911.89999999988, 108099.59999999992, 125644.40000000005, 133324.5, 141336.0, 147430.79999999987, 153295.20000000016, 160831.19999999995, 167804.0, 172441.50000000006, 178978.79999999984, 186624.3000000002, 193384.7999999999, 200565.0, 207545.0000000001, 214679.6999999998, 220712.80000000016, 226475.49999999985, 228093.0, 230751.60000000015, 237686.3999999999, 242533.5000000001, 246737.99999999983, 252990.5, 264131.89999999997, 270510.60000000003, 276864.89999999973, 282632.0, 286549.00000000023, 290438.39999999997, 294193.10000000003, 297035.1999999997, 299835.0, 304276.2000000002, 308672.5, 313886.4, 318284.3999999998, 319405.0, 318209.4000000002, 320959.6, 324704.50000000006, 328730.3999999998, 33

In [7]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
bid_prices=[]
for i in range(num_orders):
    new=1
    for j in range(len(bid_prices)):
        if orders[i].bid_price==bid_prices[j]:
            new=0
            break
    if new==1:
        bid_prices.append(orders[i].bid_price)
bid_prices.sort()

for j in range(len(bid_prices)):
    profit=0
    for i in range(num_orders):
        if orders[i].quantity<100:
            profit += orders[i].Profit(bid_prices[j], cost_less_100)
        else:
            profit += orders[i].Profit(bid_prices[j], cost_at_least_100)
    possible_profits.append(profit)
print(possible_profits)

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
for j in range(len(possible_profits)):
    if possible_profits[j]==max_profit:
        best_price=bid_prices[j]
        break
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = []
for i in range(num_orders):
    if orders[i].quantity<100 and orders[i].Profit(best_price,cost_less_100)>0:
        orders_to_go.append(orders[i])
    elif orders[i].quantity>100 and orders[i].Profit(best_price,cost_at_least_100)>0:
        orders_to_go.append(orders[i])

print("Si eseguiranno", len(orders_to_go),"ordini")

# ... assembla la lista degli ordini che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto
for i in range(len(orders_to_go)):
    for k in range(len(orders_to_go)-i-1):
        if orders_to_go[k].quantity<100:
            if orders_to_go[k+1].quantity<100:
                if orders_to_go[k].Profit(best_price,cost_less_100)<orders_to_go[k+1].Profit(best_price,cost_less_100):
                    orders_to_go[k], orders_to_go[k+1] = orders_to_go[k+1], orders_to_go[k]
            else:
                if orders_to_go[k].Profit(best_price,cost_less_100)<orders_to_go[k+1].Profit(best_price,cost_at_least_100):
                    orders_to_go[k], orders_to_go[k+1] = orders_to_go[k+1], orders_to_go[k]
        else:
            if orders_to_go[k+1].quantity<100:
                if orders_to_go[k].Profit(best_price,cost_at_least_100)<orders_to_go[k+1].Profit(best_price,cost_less_100):
                    orders_to_go[k], orders_to_go[k+1] = orders_to_go[k+1], orders_to_go[k]
            else:
                if orders_to_go[k].Profit(best_price,cost_at_least_100)<orders_to_go[k+1].Profit(best_price,cost_at_least_100):
                    orders_to_go[k], orders_to_go[k+1] = orders_to_go[k+1], orders_to_go[k]  

_ = [otg.Details() for otg in orders_to_go]

[-111146.09948009858, -99124.59445453009, -87996.86432543975, -77368.32522093345, -55762.75526271705, -34661.967728602576, -13751.79149558685, -3368.080280494196, 6732.466070418786, 16617.425333760577, 26523.16857162716, 35817.637633988204, 45329.35390409537, 54804.149138405075, 64181.449289802, 71836.89249156423, 80552.5687214531, 89010.39249362094, 96869.78329349402, 105087.33661493035, 122640.60303364128, 130328.0589898628, 138365.5348807629, 144504.32934145266, 150409.43409428062, 157971.83736322747, 164972.7000398694, 169675.35399755865, 176224.53762785427, 183873.78532454398, 190655.05386782638, 197843.34350576787, 204840.46086995737, 211984.3828392099, 218051.91765721494, 223834.10261493616, 225524.98504005777, 228246.3122529292, 235188.9268023753, 240053.54078398377, 244304.74188599145, 250563.2211201287, 261723.36174189555, 268110.2158241637, 274466.4348967774, 280268.4679954593, 284236.58998065983, 288201.3268759443, 291985.01089276304, 294869.7335761296, 297703.3247601522, 3

Gianmarco Baroni Ingranaggi ha ordinato 243 magliette taglia XXL a 13.2€ all'indirizzo [46.664931, 7.453308]
Alessandro Bressan IB ha ordinato 242 magliette taglia XS a 19.9€ all'indirizzo [45.152525, 8.134165]
Giulia Iacovelli Ingranaggi ha ordinato 239 magliette taglia XXL a 13.3€ all'indirizzo [45.824062, 8.276641]
Manuel Zacchei Giochi d'Azzardo ha ordinato 236 magliette taglia L a 17.5€ all'indirizzo [44.931719, 10.901902]
Alessandro Genalti Latte ha ordinato 230 magliette taglia XXL a 16.2€ all'indirizzo [45.190932, 10.15473]
Giulia Fedeli Grissini ha ordinato 230 magliette taglia XL a 19.1€ all'indirizzo [46.365271, 9.160471]
Gianmarco Baroni Azienda Agricola ha ordinato 229 magliette taglia S a 14.6€ all'indirizzo [46.002357, 10.304807]
Gianmarco Moneta Frumento ha ordinato 227 magliette taglia M a 14.4€ all'indirizzo [45.121825, 10.002931]
Alessandro Bressan Ingranaggi ha ordinato 225 magliette taglia S a 18.0€ all'indirizzo [44.462696, 8.818987]
Francesco Cerisola Giocattoli 